In [ ]:
#Installing dependencies needed

!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

#INSTRUCTIONS





*   Download the dataset and uncompresed

*   Put the uncompresed file in that directory
 
*   Create a "bigdata" folder in My drive from google

Note: Give access to colab to mount that directory copying and pasting
the access code generated





In [ ]:
ls

In [ ]:
!rm -rf spark-2.4.4-bin-hadoop2.7.tgz

In [ ]:
#setting envirenment variables
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

In [ ]:
#importing libraries
import findspark
findspark.init()
from pyspark.sql import SparkSession

In [ ]:
#start spark local session
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
#mount personal drive from google
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls "/content/drive/My Drive/bigdata/hw4/ml-1m"

In [ ]:
# location of the file in one personal drive
file_path="/content/drive/My Drive/bigdata/hw4/ml-1m"
file="/content/drive/My Drive/bigdata/hw4/ml-1m/ratings.dat"


In [ ]:
#loading dataframe
df = spark.read.format("csv").load(file)

RATINGS FILE DESCRIPTION
================================================================================

All ratings are contained in the file "ratings.dat" and are in the
following format:

UserID::MovieID::Rating::Timestamp

- UserIDs range between 1 and 6040 
- MovieIDs range between 1 and 3952
- Ratings are made on a 5-star scale (whole-star ratings only)
- Timestamp is represented in seconds since the epoch as returned by time(2)
- Each user has at least 20 ratings


In [ ]:
#importing some libraries 
from pyspark.sql.types import *
import pyspark.sql.functions as f
import time
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row
from pyspark.sql.functions import udf
from pyspark.mllib.linalg import VectorUDT
from pyspark.mllib.linalg import Vectors
from pyspark.ml.linalg import DenseVector
import numpy as np

In [ ]:
####Rating document loaded show
df_split = df.select(f.split(df[0],"::")).rdd.flatMap(lambda x: x).toDF(schema=["UserID","MovieID","Rating","TimeStamp"])
df_split.show(2)

Output 1)


In [ ]:
###########Output 1)
now = time.time()
df_split.groupBy("UserID","MovieID").agg(f.mean('Rating')).show(10)
elapsed = (time.time() - now)
print("exec time:",elapsed,"ms")

In [ ]:
#####Selecting relevant data###################
df_split=df_split.select(["UserID","MovieID","Rating"])

In [ ]:
df_split.printSchema()

In [ ]:
#Casting to numeric type
df_split=df_split.withColumn("UserID", df_split["UserID"].cast(IntegerType()))
df_split=df_split.withColumn("MovieID", df_split["MovieID"].cast(IntegerType()))
df_split=df_split.withColumn("Rating", df_split["Rating"].cast(IntegerType()))

In [ ]:
#checking final Schema
df_split.printSchema()

*Output* 2 )
User user item cosine similarity

In [ ]:
training_data, test_data = df_split.randomSplit([0.8, 0.2])

In [ ]:
#################### (UserId ,(movies, ratings))
df2= training_data.groupBy("UserID").agg(f.collect_list("MovieID").alias("Movies"),f.collect_list("Rating").alias("Ratings"))
df2=df2.sort('UserID', ascending=True)

In [ ]:
df2.show(4)

In [ ]:
SIZE=3952
def to_vector(x,y):
  return Vectors.sparse(3952, x, y)
to_vector_udf = udf(to_vector,VectorUDT())

In [ ]:
# Create (UserID, [movie1, movie2...],[rating1, rating2...] struct
df3=df2.withColumn('SparceVector',to_vector_udf('Movies', 'Ratings'))
df3.show(4)

In [ ]:
df4= df3.select("UserID","SparceVector")
df4.show(4)

In [ ]:
###Casting SparseVector to dense vector
def sparse_to_array(v):
  v = DenseVector(v)
  new_array = list([float(x) for x in v])
  return new_array
sparse_to_array_udf = udf(sparse_to_array, ArrayType(FloatType()))

In [ ]:
df5 = df4.withColumn('Features', sparse_to_array_udf('SparceVector'))

In [ ]:
df5.show(4)

In [ ]:
df6= df5.select("UserID","Features")

In [ ]:
cartesianRDD = df6.rdd.cartesian(df6.rdd)
cartesian=cartesianRDD.toDF()

In [ ]:
cartesian.show(4)

In [ ]:
def dot(x,y):
  x= np.asarray(x[1])
  y= np.asarray(y[1])
  dot= np.dot(x,y)

  norm_x = np.sqrt(np.dot(x,x))
  norm_y=  np.sqrt(np.dot(y,y))

  sim= dot/ (norm_x * norm_y)
  return float(sim)

dot_udf = udf(dot, FloatType())

In [ ]:
def makePair(x,y):
  x= np.asarray(x[0])
  y= np.asarray(y[0])
  return Vectors.dense([x,y])
makePair_udf=udf(makePair,VectorUDT())

In [ ]:
similarities=cartesian.withColumn('Pairs',makePair_udf(cartesian['_1'], cartesian['_2']))
similarities= similarities.withColumn("Sims", dot_udf(cartesian['_1'], cartesian['_2'])  )

In [ ]:
similarities.show(4)

In [ ]:
sim= similarities.select("Pairs", "Sims")

In [ ]:
#Ouput 2 ([user_i, user_j], similarity)
now = time.time()
sim=sim[sim["Sims"]!=1.0]
sim.show(10)
elapsed = (time.time() - now)
print("time for evaluating the model:",elapsed,"ms")

In [ ]:
sim_final=sim.select("Sims").distinct().sort('Sims', ascending=False)

In [ ]:
sim_final.show(10)

Output 3 ) Item- item cosine similarity

In [ ]:
# Create (UserID, [movie1, movie2...],[rating1, rating2...] struct
df_2= training_data.groupBy("MovieID").agg(f.collect_list("UserID").alias("Users"),f.collect_list("Rating").alias("Ratings"))
df_2=df_2.sort('MovieID', ascending=True)


In [ ]:
def to_vect(x,y):
  return Vectors.sparse(6040, x, y)

to_vect_udf = udf(to_vect, VectorUDT())



In [ ]:
df_3=df_2.withColumn('SparceVector',to_vect_udf('Users','Ratings'))

In [ ]:
df_3.show(4)

In [ ]:
df_4= df_3.select("MovieID","SparceVector")


In [ ]:
df_5 = df_4.withColumn('Features', sparse_to_array_udf('SparceVector'))


In [ ]:
df_6= df_5.select("MovieID","Features")

In [ ]:
df_6.show(4)

In [ ]:
cartesianRDD = df_6.rdd.cartesian(df_6.rdd)
cartesian=cartesianRDD.toDF()

In [ ]:
similarities=cartesian.withColumn('Pairs',makePair_udf(cartesian['_1'], cartesian['_2']))
similarities= similarities.withColumn("Sims", dot_udf(cartesian['_1'], cartesian['_2'])  )

In [ ]:
sim= similarities.select("Pairs", "Sims")

In [ ]:
sim=sim[sim["Sims"]!=1.0]
sim.show()

In [ ]:
sim_final=sim.select("Sims").distinct().sort('Sims', ascending=False)

Output 4 

In [ ]:

# Build the recommendation model using ALS on the training data

als = ALS(maxIter=5, regParam=0.01, userCol="UserID", itemCol="MovieID", ratingCol="Rating",
          coldStartStrategy="drop")


In [ ]:

now = time.time()
model = als.fit(training_data)
# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test_data)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="Rating",
                                predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
elapsed = (time.time() - now)
print("time for evaluating the model:",elapsed,"ms")
print("Root-mean-square error = " + str(rmse))
